In [1]:
from src.mgoptimization.hybrids import *
from src.mgoptimization.pso import *
import os

In [2]:
long = -3
lat = 24

In [3]:
pv_data_folder = ''

In [4]:
token = 'cdf20d12d762111de60af365755a3e53e6b0b719'

In [5]:
get_pv_data(lat, long, token, pv_data_folder)

In [6]:
hourly_ghi, hourly_temp = read_environmental_data(os.path.join(pv_data_folder, 'pv_data_lat_{}_long_{}.csv'.format(lat, long)))

In [7]:
demand = 365*361
tier = 2
diesel_price = 1

In [8]:
load_curve = calc_load_curve(tier, demand)

In [13]:
result = optimizer(diesel_price=diesel_price,
                   hourly_ghi=hourly_ghi,
                   hourly_temp=hourly_temp,
                   load_curve=load_curve,
                   start_year=2020,
                   end_year=2044,
                   discount_rate=0.08,
                   diesel_cost=378,  # diesel generator capital cost, USD/kW rated power
                   battery_cost=432,  # battery capital capital cost, USD/kWh of storage capacity
                   full_life_cycles=2000, #full battery cycles before failure
                   inverter_cost=608,  # USD/kW
                   pv_cost=1147,  # PV panel capital cost, USD/kW peak power
                   diesel_limit=1, # Maximum share of generation over a year coming from diesel generator
                   charge_controller=0,
                   lpsp_max=0.0,  # maximum loss of load allowed over the year
                   iterations=100,
                   n_particles=50,
                   ftol=0.0001,
                   ftol_iter=10,
                   verbose=True)

print('Generation LCOE is found to be {} USD/kWh'.format(round(result['gen_lcoe'], 2)))
print('Optimum PV capacity is found to be {} kW'.format(round(result['pv_capacity'], 1)))
print('Optimum diesel capacity is found to be {} kW'.format(round(result['diesel_gen_capacity'], 1)))
print('Optimum battery capacity is found to be {} kWh'.format(round(result['battery_capacity'], 1)))
print('NPC of optimum configuration is {} USD'.format(round(result['npc'], 0)))
print('Battery life of optimum configuration is {} years'.format(round(result['battery_life'], 0)))

2023-10-25 10:25:20,076 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:  83%|██████████████████████████████████████████████████▋          |83/100, best_cost=0.295
2023-10-25 10:25:24,059 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2946394499026899, best pos: [ 72.13927692 221.89663182   1.14851694]


Generation LCOE is found to be 0.29 USD/kWh
Optimum PV capacity is found to be 72.1 kW
Optimum diesel capacity is found to be 1.1 kW
Optimum battery capacity is found to be 221.9 kWh
NPC of optimum configuration is 429227.0 USD
Battery life of optimum configuration is 6 years


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
load = np.array(load_curve[2400:2448])
pv_gen = np.array(result['hourly_pv_gen'][2400:2448])
diesel_gen = np.array(result['hourly_diesel_gen'][2400:2448])
battery_soc = np.array(result['battery_hourly_soc'][2400:2448])
battery_usage = np.array(result['hourly_battery_usage'][2400:2448])

plt.plot(load, label='Load')
plt.plot(pv_gen, label='PV gen')
plt.plot(diesel_gen, 'o', label='Diesel gen')
plt.plot(battery_soc, label='Battery SOC')
plt.plot(battery_usage, 'o', label='Battery discharge')
plt.legend(loc='lower right')